### Step 1 & 2: Initializing and generating historical data

In [1]:
import numpy as np
import pandas as pd
from scipy.special import logit, expit
from scipy.stats import bernoulli, multivariate_normal
from sklearn.linear_model import LogisticRegression

class AuctionData:
    def __init__(self):
        self.alpha = None
        self.beta = None
        self.gamma = None
        self.pCTR_biased = None

    def generate_parameters(self, num_features):
        self.alpha = np.random.normal(0.1, 1, num_features)
        self.beta = np.random.normal(0.1, 1, num_features)
        self.gamma = np.random.normal(0.1, 1, num_features)
        #A = np.random.normal(0, 1, (num_features + 1, num_features + 1))
        #self.Sigma = np.dot(A, A.T)

    def sample_reparameterized_beta(self, mu, phi, size):
        mu = np.clip(mu, 0.01, 0.99)
        alpha = mu * phi
        beta_param = (1 - mu) * phi
        return np.random.beta(alpha, beta_param, size=size)

    def initialize_training_data(self, num_records_initial, num_features):
        X_k = []
        eta_k = []
        y_k = []
        D_k = []
        
        while len(D_k) < 5000:
            X_binary = np.random.binomial(1, 0.5, (num_records_initial, 10))
            X_uniform_5 = np.random.uniform(-5, 5, (num_records_initial, 10))
            X_uniform_2 = np.random.uniform(-2, 2, (num_records_initial, 5))
            X_tmp = np.hstack((X_binary, X_uniform_5, X_uniform_2))
            eta_tmp = np.random.uniform(-5, 5, num_records_initial)
            
            D_tmp_prob = expit(np.dot(X_tmp, self.gamma) + eta_tmp)
            D_tmp = bernoulli.rvs(D_tmp_prob)
            
            X_tmp = X_tmp[D_tmp == 1]
            eta_tmp = eta_tmp[D_tmp == 1]
            p_tmp = expit(np.dot(X_tmp, self.beta) + eta_tmp)
            y_tmp = bernoulli.rvs(p_tmp)
            D_tmp = D_tmp[D_tmp == 1]
            
            X_k.extend(X_tmp)
            eta_k.extend(eta_tmp)
            y_k.extend(y_tmp)
            D_k.extend(D_tmp)

        return np.array(X_k)[:5000], np.array(eta_k)[:5000], np.array(y_k)[:5000], np.array(D_k)[:5000]

    def train_logistic_regression(self, X_k, y_k):
        self.pCTR_biased = LogisticRegression()
        self.pCTR_biased.fit(X_k, y_k)

    def generate_historical_auction_data(self, num_auctions, num_auctioneers_per_auction, num_features):
        historical_data = []
        for _ in range(num_auctions):
            X_binary = np.random.binomial(1, 0.5, (num_auctioneers_per_auction, 10))
            X_uniform_5 = np.random.uniform(-5, 5, (num_auctioneers_per_auction, 10))
            X_uniform_2 = np.random.uniform(-2, 2, (num_auctioneers_per_auction, 5))
            X_j = np.hstack((X_binary, X_uniform_5, X_uniform_2))

            eta_j = np.random.uniform(-5, 5, num_auctioneers_per_auction)#np.random.normal(0, 1, num_auctioneers_per_auction)
            mu_j = expit(np.dot(X_j, self.alpha))
            bids = self.sample_reparameterized_beta(mu_j, 2, size=num_auctioneers_per_auction)
            pCTR_j = self.pCTR_biased.predict_proba(X_j)[:, 1]
            auction_scores = bids * pCTR_j
            j_star = np.argmax(auction_scores)
            p_j = expit(np.dot(X_j[j_star], self.beta) + eta_j[j_star])
            y_j = np.zeros(num_auctioneers_per_auction)
            y_j[j_star] = bernoulli.rvs(p_j)
            D_j = np.zeros(num_auctioneers_per_auction)
            D_j[j_star] = 1
            historical_data.append((y_j, X_j, bids, D_j))

        return historical_data

    def flatten_historical_data(self, historical_data):
        y_flat = []
        X_flat = []
        bids_flat = []
        D_flat = []

        for y_j, X_j, bids, D_j in historical_data:
            y_flat.extend(y_j)
            X_flat.extend(X_j)
            bids_flat.extend(bids)
            D_flat.extend(D_j)

        y_flat = np.array(y_flat)
        X_flat = np.array(X_flat)
        bids_flat = np.array(bids_flat)
        D_flat = np.array(D_flat)

        return y_flat, X_flat, bids_flat, D_flat

    def generate_and_train(self, num_records_initial=5000, num_auctions=5000, num_auctioneers_per_auction=20, num_features=25, seed=None):
        # Set the seed for this replication
        if seed is not None:
            np.random.seed(seed)
            tf.random.set_seed(seed)
        
        # Generate parameters
        self.generate_parameters(num_features)

        # Step 1: Initializing and generating training data
        X_k, eta_k, y_k, D_k = self.initialize_training_data(num_records_initial, num_features)

        # Define a logistic regression model for pCTR and train it
        self.train_logistic_regression(X_k, y_k)

        # Step 2: Generating historical auction data
        historical_data = self.generate_historical_auction_data(num_auctions, num_auctioneers_per_auction, num_features)

        # Flatten the historical data for learning
        y_flat, X_flat, bids_flat, D_flat = self.flatten_historical_data(historical_data)

        return y_flat, X_flat, bids_flat, D_flat, self.alpha, self.beta, self.gamma

# Usage example
#auction_data = AuctionData()
#y_flat, X_flat, bids_flat, D_flat, alpha, beta, gamma = auction_data.generate_and_train()

### Step 3: Define and learn the baselines

In [2]:
import numpy as np
import tensorflow as tf

class Naive:
    def __init__(self):
        self.model = None

    def prepare_inputs(self, X, bids):
        inputs = {f'X{i+1}': X[:, i].reshape(-1, 1) for i in range(X.shape[1])}
        inputs['z'] = bids.reshape(-1, 1)
        return inputs

    def create_model(self, input_shape):
        inputs = {f'X{i+1}': tf.keras.layers.Input(shape=(1,), name=f'X{i+1}') for i in range(input_shape[1])}
        inputs_iv = {"z": tf.keras.layers.Input(shape=(1,), name="z")}
        inputs_with_iv = {**inputs, **inputs_iv}

        input_net = tf.keras.layers.Concatenate(axis=-1)(list(inputs.values()))
        input_net = tf.keras.layers.BatchNormalization()(input_net)
        input_net_iv = tf.keras.layers.Concatenate(axis=-1)([input_net, inputs_iv['z']])

        pCTR = tf.keras.layers.Dense(256, activation="swish")(input_net_iv)
        pCTR = tf.keras.layers.BatchNormalization()(pCTR)
        pCTR = tf.keras.layers.Dense(256, activation="relu")(pCTR)
        pCTR = tf.keras.layers.BatchNormalization()(pCTR)
        pCTR = tf.keras.layers.Dense(256, activation="relu")(pCTR)
        pCTR = tf.keras.layers.BatchNormalization()(pCTR)
        pCTR = tf.keras.layers.Dense(1, activation="sigmoid", name="click")(pCTR)
        model = tf.keras.Model(inputs=list(inputs_with_iv.values()), outputs=pCTR)

        model.compile(
            optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
            loss="binary_crossentropy",
        )
        return model

    def train_model(self, X_flat, bids_flat, y_flat, D_flat, epochs=40, steps_per_epoch=1000):
        inputs_for_nn = self.prepare_inputs(X_flat, bids_flat)
        self.model = self.create_model(X_flat.shape)

        cp_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath="checkpoints/naive_checkpoint",
            save_best_only=True,
            verbose=1,
            monitor="loss",
        )
        es_callback = tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            patience=3,
            verbose=1
        )

        self.model.fit(
            {**inputs_for_nn},
            y_flat,
            sample_weight=D_flat,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            callbacks=[cp_callback, es_callback]
        )
        return self.model

# Usage example
#naive_instance = Naive()
#naive_model = naive_instance.train_model(X_flat, bids_flat, y_flat, D_flat)

/Users/emo_157/Desktop/24_coldstart/sim/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
import numpy as np
import tensorflow as tf

class IVBS:
    def __init__(self):
        self.model = None

    def prepare_inputs(self, X, bids):
        inputs = {f'X{i+1}': X[:, i].reshape(-1, 1) for i in range(X.shape[1])}
        inputs['z'] = bids.reshape(-1, 1)
        return inputs

    def create_model(self, input_shape):
        inputs = {f'X{i+1}': tf.keras.layers.Input(shape=(1,), name=f'X{i+1}') for i in range(input_shape[1])}
        inputs_iv = {"z": tf.keras.layers.Input(shape=(1,), name="z")}
        inputs_with_iv = {**inputs, **inputs_iv}

        inputs_net = tf.keras.layers.Concatenate(axis=-1)(list(inputs.values()))
        inputs_iv_net = list(inputs_iv.values())[0]
        inputs_net_iv = tf.keras.layers.Concatenate(axis=-1)([inputs_net, inputs_iv_net])

        pIMP_iv = tf.keras.layers.Dense(128, activation="swish")(inputs_net_iv)
        pIMP_iv = tf.keras.layers.BatchNormalization()(pIMP_iv)
        pIMP_iv = tf.keras.layers.Dense(1, activation="sigmoid", name="impression_iv")(pIMP_iv)
        pIMP_iv_model = tf.keras.Model(inputs=inputs_with_iv, outputs=pIMP_iv)

        model = tf.keras.Model(inputs=inputs_with_iv, outputs=pIMP_iv_model.output)
        model.compile(
            optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
            loss="binary_crossentropy",
        )
        return model

    def train_model(self, X_flat, bids_flat, y_flat, D_flat, epochs=40, steps_per_epoch=1000):
        inputs_for_nn = self.prepare_inputs(X_flat, bids_flat)
        self.model = self.create_model(X_flat.shape)

        cp_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath="checkpoints/IV-BS_checkpoint",
            save_best_only=True,
            verbose=1,
            monitor="loss",
        )
        es_callback = tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            patience=3,
            verbose=1
        )

        self.model.fit(
            {**inputs_for_nn},
            y_flat,
            sample_weight=D_flat,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            callbacks=[cp_callback, es_callback]
        )
        return self.model

# Usage example
#IV-BS_instance = IV-BS()
#ivbs_model = ivbs_instance.train_model(X_flat, bids_flat, y_flat, D_flat)

In [4]:
import numpy as np
import tensorflow as tf

class IPS:
    def __init__(self):
        self.pIMP_model = None
        self.pCTR_model = None

    def prepare_inputs(self, X, bids):
        inputs = {f'X{i+1}': X[:, i].reshape(-1, 1) for i in range(X.shape[1])}
        inputs['z'] = bids.reshape(-1, 1)
        return inputs

    def create_pIMP_model(self, input_shape):
        inputs = {f'X{i+1}': tf.keras.layers.Input(shape=(1,), name=f'X{i+1}') for i in range(input_shape[1])}
        inputs_iv = {"z": tf.keras.layers.Input(shape=(1,), name="z")}
        inputs_with_iv = {**inputs, **inputs_iv}

        inputs_net = tf.keras.layers.Concatenate(axis=-1)(list(inputs.values()))
        inputs_iv_net = list(inputs_iv.values())[0]
        inputs_net_iv = tf.keras.layers.Concatenate(axis=-1)([inputs_net, inputs_iv_net])

        pIMP_iv = tf.keras.layers.Dense(128, activation="swish")(inputs_net_iv)
        pIMP_iv = tf.keras.layers.BatchNormalization()(pIMP_iv)
        pIMP_iv = tf.keras.layers.Dense(1, activation="sigmoid", name="impression_iv")(pIMP_iv)
        pIMP_iv_model = tf.keras.Model(inputs=inputs_with_iv, outputs=pIMP_iv)

        pIMP_iv_model.compile(
            optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
            loss="binary_crossentropy",
        )

        return pIMP_iv_model

    def train_pIMP_model(self, X_flat, bids_flat, y_flat, epochs=50, steps_per_epoch=1000):
        inputs_for_nn = self.prepare_inputs(X_flat, bids_flat)
        self.pIMP_model = self.create_pIMP_model(X_flat.shape)

        cp_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath="checkpoints/ipsimp_checkpoint",
            save_best_only=True,
            verbose=1,
            monitor="loss",
        )
        es_callback = tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            patience=3,
            verbose=1
        )

        self.pIMP_model.fit(inputs_for_nn, y_flat, epochs=epochs, steps_per_epoch=steps_per_epoch, callbacks=[cp_callback, es_callback])
        return self.pIMP_model

    def create_pCTR_model(self, input_shape):
        inputs = {f'X{i+1}': tf.keras.layers.Input(shape=(1,), name=f'X{i+1}') for i in range(input_shape[1])}
        inputs_net = tf.keras.layers.Concatenate(axis=-1)(list(inputs.values()))

        pCTR = tf.keras.layers.Dense(256, activation="swish")(inputs_net)
        pCTR = tf.keras.layers.BatchNormalization()(pCTR)
        pCTR = tf.keras.layers.Dense(256, activation="relu")(pCTR)
        pCTR = tf.keras.layers.BatchNormalization()(pCTR)
        pCTR = tf.keras.layers.Dense(256, activation="relu")(pCTR)
        pCTR = tf.keras.layers.BatchNormalization()(pCTR)
        pCTR = tf.keras.layers.Dense(1, activation="sigmoid", name="click")(pCTR)
        pCTR_model = tf.keras.Model(inputs=inputs, outputs=pCTR)

        pCTR_model.compile(
            optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
            loss="binary_crossentropy"
        )

        return pCTR_model

    def train_pCTR_model(self, X_flat, bids_flat, y_flat, epochs=10, steps_per_epoch=1000):
        inputs_for_nn = self.prepare_inputs(X_flat, bids_flat)
        self.pCTR_model = self.create_pCTR_model(X_flat.shape)

        cp_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath="checkpoints/ipsclk_checkpoint",
            save_best_only=True,
            verbose=1,
            monitor="loss",
        )
        es_callback = tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            patience=3,
            verbose=1
        )

        # Load the pIMP model and predict the impression probabilities
        ipsimp = tf.keras.models.load_model("checkpoints/ipsimp_checkpoint")
        ips_weight = ipsimp.predict(inputs_for_nn, batch_size=100000, verbose=1).flatten()

        # Train the pCTR model with the impression weights
        self.pCTR_model.fit(inputs_for_nn, y_flat, sample_weight=ips_weight, epochs=epochs, steps_per_epoch=steps_per_epoch, callbacks=[cp_callback, es_callback])
        return self.pCTR_model

# Usage example
#ips_instance = IPS()
#ips_instance.train_pIMP_model(X_flat, bids_flat, y_flat)
#ips_model = ips_instance.train_pCTR_model(X_flat, bids_flat, y_flat)

In [5]:
import numpy as np
import tensorflow as tf

class UBIPS:
    def __init__(self):
        self.model = None

    def prepare_inputs(self, X, bids):
        inputs = {f'X{i+1}': X[:, i].reshape(-1, 1) for i in range(X.shape[1])}
        inputs['z'] = bids.reshape(-1, 1)
        return inputs

    def create_model(self, input_shape):
        inputs = {f'X{i+1}': tf.keras.layers.Input(shape=(1,), name=f'X{i+1}') for i in range(input_shape[1])}
        inputs_iv = {"z": tf.keras.layers.Input(shape=(1,), name="z")}
        inputs_with_iv = {**inputs, **inputs_iv}

        input_net = tf.keras.layers.Concatenate(axis=-1)(list(inputs.values()))
        input_net = tf.keras.layers.BatchNormalization()(input_net)
        input_net_iv = tf.keras.layers.Concatenate(axis=-1)([input_net, inputs_iv['z']])

        pIMP_iv = tf.keras.layers.Dense(128, activation="swish")(input_net_iv)
        pIMP_iv = tf.keras.layers.BatchNormalization()(pIMP_iv)
        pIMP_iv = tf.keras.layers.Dense(1, activation="sigmoid", name="impression_iv")(pIMP_iv)
        pIMP_iv_model = tf.keras.Model(inputs=inputs_with_iv, outputs=pIMP_iv)

        pCTR = tf.keras.layers.Dense(256, activation="swish")(input_net)
        pCTR = tf.keras.layers.BatchNormalization()(pCTR)
        pCTR = tf.keras.layers.Dense(256, activation="relu")(pCTR)
        pCTR = tf.keras.layers.BatchNormalization()(pCTR)
        pCTR = tf.keras.layers.Dense(256, activation="relu")(pCTR)
        pCTR = tf.keras.layers.BatchNormalization()(pCTR)
        pCTR = tf.keras.layers.Dense(1, activation="sigmoid", name="click")(pCTR)
        pCTR_model = tf.keras.Model(inputs=inputs_with_iv, outputs=pCTR)

        model = tf.keras.Model(
            inputs=inputs_with_iv,
            outputs=[pCTR_model.output * pIMP_iv_model.output]
        )
        model.compile(
            optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
            loss="binary_crossentropy",
        )

        return model

    def train_model(self, X_flat, bids_flat, y_flat, D_flat, epochs=50, steps_per_epoch=1000):
        inputs_for_nn = self.prepare_inputs(X_flat, bids_flat)
        self.model = self.create_model(X_flat.shape)

        cp_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath="checkpoints/ubips_checkpoint",
            save_best_only=True,
            verbose=1,
            monitor="loss",
        )
        es_callback = tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            patience=7,
            verbose=1
        )

        self.model.fit(
            {**inputs_for_nn},
            [y_flat],
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            callbacks=[cp_callback, es_callback]
        )
        return self.model

# Usage example
#ubips_instance = UBIPS()
#ubips_model = ubips_instance.train_model(X_flat, bids_flat, y_flat, D_flat)

### Step 4: Validating baselines with independently displayed data

In [6]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import log_loss, roc_auc_score
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.special import expit
from scipy.stats import bernoulli
import os

class ModelEvaluation:
    def __init__(self, replication_times=20, num_auctions=5000):
        self.replication_times = replication_times
        self.num_auctions = num_auctions
        self.results = {
            'Naive': {'LogLoss': [], 'AUC': []},
            'IV-BS': {'LogLoss': [], 'AUC': []},
            'UBIPS': {'LogLoss': [], 'AUC': []}
        }
        self.checkpoints_dir = "bs_checkpoints"
        self.quantile_results = {
            'Naive': {'LogLoss': [], 'AUC': []},
            'IV-BS': {'LogLoss': [], 'AUC': []},
            'UBIPS': {'LogLoss': [], 'AUC': []}
        }

        os.makedirs(self.checkpoints_dir, exist_ok=True)
        os.makedirs("res", exist_ok=True)
        os.makedirs("figs", exist_ok=True)

    def evaluate_model(self, auction_data):
        for iteration in tqdm(range(self.replication_times)):
            # Generate training data and train models
            y_flat, X_flat, bids_flat, D_flat, alpha, beta, gamma = auction_data.generate_and_train()

            naive = Naive()
            naive_model = naive.train_model(X_flat, bids_flat, y_flat, D_flat)
            naive_model.save(f"{self.checkpoints_dir}/naive_model_{iteration}.h5")

            ivbs = IVBS()
            ivbs_model = ivbs.train_model(X_flat, bids_flat, y_flat, D_flat)
            ivbs_model.save(f"{self.checkpoints_dir}/ivbs_model_{iteration}.h5")

            ubips = UBIPS()
            ubips_model = ubips.train_model(X_flat, bids_flat, y_flat, D_flat)
            ubips_model.save(f"{self.checkpoints_dir}/ubips_model_{iteration}.h5")

            # Generate validation data
            num_records_validation = 50000
            X_binary = np.random.binomial(1, 0.5, (num_records_validation, 10))
            X_uniform_5 = np.random.uniform(-5, 5, (num_records_validation, 10))
            X_uniform_2 = np.random.uniform(-2, 2, (num_records_validation, 5))
            X_l = np.hstack((X_binary, X_uniform_5, X_uniform_2))
            eta_l = np.random.uniform(-5, 5, num_records_validation)
            mu_l = expit(np.dot(X_l, alpha))
            bids = auction_data.sample_reparameterized_beta(mu_l, 2, size=num_records_validation)
            p_l = expit(np.dot(X_l, beta) + eta_l)
            y_l = bernoulli.rvs(p_l)
            inputs_for_validation = naive.prepare_inputs(X_l, bids)

            # Save validation data
            np.savez(f"{self.checkpoints_dir}/validation_data_{iteration}.npz", X_l=X_l, bids=bids, y_l=y_l)

            # Load the models
            naive = tf.keras.models.load_model(f"{self.checkpoints_dir}/naive_model_{iteration}.h5")
            ivbs = tf.keras.models.load_model(f"{self.checkpoints_dir}/ivbs_model_{iteration}.h5")
            ubips = tf.keras.models.load_model(f"{self.checkpoints_dir}/ubips_model_{iteration}.h5")

            # Naive model prediction
            naive_predictions = naive.predict(inputs_for_validation).flatten()
            naive_logloss = log_loss(y_l, naive_predictions)
            naive_auc = roc_auc_score(y_l, naive_predictions)

            # IV-BS model prediction
            ivbs_predictions = ivbs.predict(inputs_for_validation)
            if isinstance(ivbs_predictions, list):
                ivbs_predictions = ivbs_predictions[0].flatten()
            else:
                ivbs_predictions = ivbs_predictions.flatten()
            ivbs_logloss = log_loss(y_l, ivbs_predictions)
            ivbs_auc = roc_auc_score(y_l, ivbs_predictions)

            # UBIPS model prediction
            ubips_predictions = ubips.predict(inputs_for_validation)
            if isinstance(ubips_predictions, list):
                ubips_predictions = ubips_predictions[0].flatten()
            else:
                ubips_predictions = ubips_predictions.flatten()
            ubips_logloss = log_loss(y_l, ubips_predictions)
            ubips_auc = roc_auc_score(y_l, ubips_predictions)

            # Store results
            self.results['Naive']['LogLoss'].append(naive_logloss)
            self.results['Naive']['AUC'].append(naive_auc)

            self.results['IV-BS']['LogLoss'].append(ivbs_logloss)
            self.results['IV-BS']['AUC'].append(ivbs_auc)

            self.results['UBIPS']['LogLoss'].append(ubips_logloss)
            self.results['UBIPS']['AUC'].append(ubips_auc)

            # Quantile validation
            filtered_dfs = self.create_filtered_dfs(X_l, bids, eta_l, y_l)
            quantile_results = self.evaluate_quantiles(filtered_dfs, naive, ivbs, ubips)

            for model in self.results.keys():
                for metric in ['LogLoss', 'AUC']:
                    self.quantile_results[model][metric].extend(quantile_results[model][metric])

            self.save_results()

    def save_results(self):
        np.save("res/results.npy", self.results)
        np.save("res/quantile_results.npy", self.quantile_results)

    def load_results(self):
        self.results = np.load("res/results.npy", allow_pickle=True).item()
        self.quantile_results = np.load("res/quantile_results.npy", allow_pickle=True).item()

    def plot_results(self):
        models = ['Naive', 'IV-BS', 'UBIPS']
        metrics = ['AUC', 'LogLoss']
        colors = ['#1f77b4', '#ff7f0e', '#8c564b']

        data = {metric: [] for metric in metrics}
        for metric in metrics:
            for model in models:
                data[metric].append(self.results[model][metric])

        plt.figure(figsize=(15, 10))
        for i, metric in enumerate(metrics):
            plt.subplot(1, 2, i + 1)
            plt.boxplot(data[metric], labels=models, patch_artist=True,
                        boxprops=dict(facecolor=colors[i], color=colors[i]),
                        medianprops=dict(color='black'),
                        whiskerprops=dict(color=colors[i]),
                        capprops=dict(color=colors[i]),
                        flierprops=dict(markerfacecolor=colors[i], markeredgecolor=colors[i]))
            plt.title(metric)
            plt.xlabel('Model')
            #plt.ylabel(metric)
        plt.savefig("figs/results_boxplot.pdf")
        plt.close()

    def evaluate_saved_models(self):
        models = ['Naive', 'IV-BS', 'UBIPS']
        for iteration in range(self.replication_times):
            # Load validation data
            data = np.load(f"{self.checkpoints_dir}/validation_data_{iteration}.npz")
            X_l = data['X_l']
            bids = data['bids']
            y_l = data['y_l']
            inputs_for_validation = Naive().prepare_inputs(X_l, bids)

            results = {model: {'LogLoss': None, 'AUC': None} for model in models}
            for model_name in models:
                model = tf.keras.models.load_model(f"{self.checkpoints_dir}/{model_name.lower()}_model_{iteration}.h5")
                predictions = model.predict(inputs_for_validation).flatten()
                LogLoss = log_loss(y_l, predictions)
                AUC = roc_auc_score(y_l, predictions)

                results[model_name]['LogLoss'] = LogLoss
                results[model_name]['AUC'] = AUC

                self.results[model_name]['LogLoss'].append(LogLoss)
                self.results[model_name]['AUC'].append(AUC)

            self.save_results()

    def plot_intermediate_results(self):
        try:
            self.load_results()
        except:
            pass

        self.plot_results()
        self.plot_quantile_results()

    # Functions for quantile validation
    def create_filtered_dfs(self, X_l, bids, eta_l, y_l):
        filtered_dfs = {}
        for lower_quantile in range(5, 51, 5):
            upper_quantile = 100 - lower_quantile

            lower_bound = np.percentile(eta_l, lower_quantile)
            upper_bound = np.percentile(eta_l, upper_quantile)

            relevant_indices = np.where((eta_l <= lower_bound) | (eta_l >= upper_bound))[0]

            filtered_dfs[f"{lower_quantile * 2}"] = {
                'X': X_l[relevant_indices],
                'bids': bids[relevant_indices],
                'y': y_l[relevant_indices]
            }

        return filtered_dfs

    def evaluate_quantiles(self, filtered_dfs, naive, ivbs, ubips):
        results = {
            'Naive': {'LogLoss': [], 'AUC': []},
            'IV-BS': {'LogLoss': [], 'AUC': []},
            'UBIPS': {'LogLoss': [], 'AUC': []}
        }

        for key, data in filtered_dfs.items():
            X_filtered = data['X']
            bids_filtered = data['bids']
            y_filtered = data['y']

            # Evaluate Naive model
            naive_logloss, naive_auc = self.evaluate_model_on_quantile(X_filtered, bids_filtered, y_filtered, naive)
            results['Naive']['LogLoss'].append(naive_logloss)
            results['Naive']['AUC'].append(naive_auc)

            # Evaluate IVBS model
            ivbs_logloss, ivbs_auc = self.evaluate_model_on_quantile(X_filtered, bids_filtered, y_filtered, ivbs)
            results['IV-BS']['LogLoss'].append(ivbs_logloss)
            results['IV-BS']['AUC'].append(ivbs_auc)

            # Evaluate UBIPS model
            ubips_logloss, ubips_auc = self.evaluate_model_on_quantile(X_filtered, bids_filtered, y_filtered, ubips)
            results['UBIPS']['LogLoss'].append(ubips_logloss)
            results['UBIPS']['AUC'].append(ubips_auc)

        return results

    def evaluate_model_on_quantile(self, X_filtered, bids_filtered, y_filtered, model):
        inputs_for_eval = Naive().prepare_inputs(X_filtered, bids_filtered)
        predictions = model.predict(inputs_for_eval)
        if isinstance(predictions, list):
            predictions = predictions[0].flatten()
        else:
            predictions = predictions.flatten()

        LogLoss = log_loss(y_filtered, predictions)
        AUC = roc_auc_score(y_filtered, predictions)

        return LogLoss, AUC

    def plot_quantile_results(self):
        metrics = ["AUC", "LogLoss"]
        model_order = ['Naive', 'IV-BS', 'UBIPS']
        colors = {
            'Naive': '#1f77b4',
            'IV-BS': '#ff7f0e',
            'UBIPS': '#8c564b',
        }

        quantiles = sorted(list(self.create_filtered_dfs(np.zeros((10, 10)), np.zeros(10), np.zeros(10), np.zeros(10)).keys()), key=lambda x: int(x))

        for metric in metrics:
            fig, ax1 = plt.subplots(figsize=(18, 10))
            positions = np.arange(len(quantiles))

            # Boxplot for each model
            for model, color in zip(model_order, colors.values()):
                data = [self.quantile_results[model][metric][i::len(quantiles)] for i in range(len(quantiles))]
                bp = ax1.boxplot(
                    data,
                    positions=positions,
                    widths=0.6,
                    patch_artist=True,
                    boxprops=dict(facecolor=color, color=color),
                    medianprops=dict(color='black'),
                    whiskerprops=dict(color=color),
                    capprops=dict(color=color),
                    flierprops=dict(markerfacecolor=color, markeredgecolor=color)
                )
                for box in bp['boxes']:
                    box.set_label(model)

            ax1.set_xlabel("Outside Quantiles of $\eta_l$ in Users' Click Response", fontsize=20)
            ax1.set_ylabel(metric, fontsize=20)
            ax1.set_xticks(positions)
            ax1.set_xticklabels(quantiles, fontsize=20)
            ax1.tick_params(axis='y', labelsize=20)

            # Calculate relative improvement
            if metric == "AUC":
                baseline = np.array([self.quantile_results['Naive'][metric][i::len(quantiles)] for i in range(len(quantiles))])
                improvements = {model: [] for model in model_order if model != 'Naive'}
                for model in improvements.keys():
                    model_data = np.array([self.quantile_results[model][metric][i::len(quantiles)] for i in range(len(quantiles))])
                    relative_improvement = ((model_data - 0.5) / (baseline - 0.5) - 1) * 100
                    improvements[model] = relative_improvement
            elif metric == "LogLoss":
                baseline = np.array([self.quantile_results['Naive'][metric][i::len(quantiles)] for i in range(len(quantiles))])
                improvements = {model: [] for model in model_order if model != 'Naive'}
                for model in improvements.keys():
                    model_data = np.array([self.quantile_results[model][metric][i::len(quantiles)] for i in range(len(quantiles))])
                    relative_improvement = (baseline - model_data) / baseline * 100
                    improvements[model] = relative_improvement

            # Plot relative improvement
            ax2 = ax1.twinx()
            for model, color in zip([m for m in model_order if m != 'Naive'], [colors[m] for m in model_order if m != 'Naive']):
                mean_improvement = np.mean(improvements[model], axis=1)
                std_improvement = np.std(improvements[model], axis=1)
                ax2.plot(positions, mean_improvement, color=color, marker='o', label=f'Relative {model}')
                ax2.fill_between(positions, mean_improvement - std_improvement, mean_improvement + std_improvement, color=color, alpha=0.2)
            
            # Add Naive model's relative improvement (always 0%)
            ax2.axhline(0, color=colors['Naive'], linestyle='--', label='Relative Naive')

            ax2.set_ylabel(f'Relative {metric} Improvement (%)', fontsize=20)
            ax2.tick_params(axis='y', labelsize=20)
            
            legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=model) for model, color in colors.items()]
            ax1.legend(handles=legend_elements, loc='best', fontsize=14)

            plt.tight_layout()
            plt.savefig(f"figs/sim_{metric}.pdf", bbox_inches='tight')
            plt.close()

# Usage example
evaluation = ModelEvaluation(replication_times=20)
#evaluation.evaluate_model(AuctionData())
#evaluation.plot_results()
#evaluation.plot_quantile_results()
evaluation.plot_intermediate_results()

/var/folders/qb/31p7jmc96_n0fns67992g7x00000gn/T/ipykernel_54931/1861601639.py:132: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data[metric], labels=models, patch_artist=True,
/var/folders/qb/31p7jmc96_n0fns67992g7x00000gn/T/ipykernel_54931/1861601639.py:132: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data[metric], labels=models, patch_artist=True,
